<a href="https://colab.research.google.com/github/FreddyPinto/recsys-steam-games/blob/feature/notebooks/1.1-etl-steam-games.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#ETL dataset `steam_games.gz.json`

En este notebook, nuestro objetivo es realizar el proceso de extracción, transformación y carga (ETL) de los datos de los juegos disponibles en Steam, para poder disponer de ellos mediante una API. Este proceso nos permitirá acceder a la información de los juegos de forma estructurada y actualizada, así como preparar los datos para su posterior análisis y modelado.

## 0 Configuraciones Globales e Importaciones

En esta sección, importamos todas las bibliotecas y modulos necesarios para nuestro proceso ETL y establecemos configuraciones globales de ser requerido.

### Importación de Módulos

In [47]:
import sys
import os
import pandas as pd
import numpy as np
import gzip
import json
import re

print(f"System version: {sys.version}")
print(f"Pandas version: {pd.__version__}")
print(f"Numpy version: {np.__version__}")
print(f"json version: {json.__version__}")
print(f"re version: {re.__version__}")

System version: 3.10.12 (main, Jun 11 2023, 05:26:28) [GCC 11.4.0]
Pandas version: 1.5.3
Numpy version: 1.23.5
json version: 2.0.9
re version: 2.2.1


## 1 Extracción

En esta sección, extraemos los datos del archivo`steam_games.gz.json` y describimos a detalle su contenido.

### 1.1 Extracción de los datos

Se extraen los datos descomprimiendo el archivo con el módulo `gzip`, leemos línea por línea el JSON con el módulo `json`, almacenamos en una lista de `Python` y cargamos a un Dataframe de `pandas` para observar su contenido.

In [2]:
# Ruta al dataset
path = 'data/raw/steam_games.json.gz'

# Descomprimimos el archivo con gzip y leemos línea por línea el JSON.
data = []
with gzip.open(path, 'r') as f:
    for line in f:
      data.append(json.loads(line))

# Convertimos a DataFrame
df_games = pd.DataFrame(data)
df_games

,publisher,genres,app_name,title,url,release_date,tags,reviews_url,specs,price,early_access,id,developer
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...
120440,Ghost_RUS Games,"[Casual, Indie, Simulation, Strategy]",Colony On Mars,Colony On Mars,http://store.steampowered.com/app/773640/Colon...,2018-01-04,"[Strategy, Indie, Casual, Simulation]",http://steamcommunity.com/app/773640/reviews/?...,"[Single-player, Steam Achievements]",1.99,False,773640,"Nikita ""Ghost_RUS"""
120441,Sacada,"[Casual, Indie, Strategy]",LOGistICAL: South Africa,LOGistICAL: South Africa,http://store.steampowered.com/app/733530/LOGis...,2018-01-04,"[Strategy, Indie, Casual]",http://steamcommunity.com/app/733530/reviews/?...,"[Single-player, Steam Achievements, Steam Clou...",4.99,False,733530,Sacada
120442,Laush Studio,"[Indie, Racing, Simulation]",Russian Roads,Russian Roads,http://store.steampowered.com/app/610660/Russi...,2018-01-04,"[Indie, Simulation, Racing]",http://steamcommunity.com/app/610660/reviews/?...,"[Single-player, Steam Achievements, Steam Trad...",1.99,False,610660,Laush Dmitriy Sergeevich
120443,SIXNAILS,"[Casual, Indie]",EXIT 2 - Directions,EXIT 2 - Directions,http://store.steampowered.com/app/658870/EXIT_...,2017-09-02,"[Indie, Casual, Puzzle, Singleplayer, Atmosphe...",http://steamcommunity.com/app/658870/reviews/?...,"[Single-player, Steam Achievements, Steam Cloud]",4.99,False,658870,"xropi,stev3ns"


- Hacemos un resumen conciso del Dataframe para observar los tipos de datos por columnas, verificar nulos, el número de valores únicos, la moda y su frecuencia.

In [3]:
df_games.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 120445 entries, 0 to 120444
Data columns (total 13 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   publisher     24083 non-null  object
 1   genres        28852 non-null  object
 2   app_name      32133 non-null  object
 3   title         30085 non-null  object
 4   url           32135 non-null  object
 5   release_date  30068 non-null  object
 6   tags          31972 non-null  object
 7   reviews_url   32133 non-null  object
 8   specs         31465 non-null  object
 9   price         30758 non-null  object
 10  early_access  32135 non-null  object
 11  id            32133 non-null  object
 12  developer     28836 non-null  object
dtypes: object(13)
memory usage: 11.9+ MB


In [4]:
df_games.describe()

,publisher,genres,app_name,title,url,release_date,tags,reviews_url,specs,price,early_access,id,developer
count,24083,28852,32133,30085,32135,30068,31972,32133,31465,30758.00,32135,32133,28836
unique,8239,883,32094,30054,32135,3582,15395,32132,4649,162.00,2,32132,10992
top,Ubisoft,[Action],Soundtrack,Soundtrack,http://store.steampowered.com/app/761140/Lost_...,2012-10-16,"[Casual, Simulation]",http://steamcommunity.com/app/612880/reviews/?...,[Single-player],4.99,False,612880,Ubisoft - San Francisco
freq,385,1880,3,3,1,100,1292,2,2794,4278.00,30188,2,1259


### 1.2 Descripción del los datos.

A partir del análisis exploratorio anterior, podemos observar que el conjunto de datos contiene **120445** registros y **13** variables con información sobre juegos de Steam. Sin embargo, hay una gran cantidad de datos nulos. A continuación, se describe las variables:

- **publisher**: empresa publicadora del juego.
- **genres**: género del juego. Esta formado por una lista de uno o mas géneros por registro.
- **app_name**: nombre del juego.
- **title**: título del juego.
- **url**: URL de publicación del juego.
- **release_date**: fecha de lanzamiento en formato 2018-01-04.
- **tags**: etiquetas del juego. Esta formado por una lista de uno o más etiquetas por registro.
- **reviews_url**: reviews del juego.
- **specs**: especificaciones. Es una lista con uno o mas string con las especificaciones.
- **price**: precio del juego.
- **early_access**: indica el acceso temprano al juego con un True/False.
- **id**: identificador único del juego.
- **developer**: desarrollador del juego.


## 2 Transformación

En esta sección, realizamos la limpieza inicial de los datos y las transformaciones necesarias. Esto puede incluir la creación de nuevas columnas a partir de las existentes, la eliminación de duplicados o columnas innecesarias, la gestión de valores nulos o la corrección de tipos de datos.

### 2.1 Columna `id`

#### 2.1.1 Gestión de valores nulos

Observamos que hay un número considerable de valores nulos que procedemos a eliminar tomando como referencia la columna `id` que contiene el identificador único de cada item.

In [5]:
df_games.dropna(subset='id', inplace=True)
df_games.reset_index(drop=True, inplace=True)
df_games.shape

(32133, 13)

#### 2.1.2 Verificación de duplicados

Del análisis exploratorio anterior notamos que el `id` **612880** esta duplicado. Vamos a comprobarlo directamente:

In [6]:
df_games[df_games['id'].duplicated(keep=False)]

,publisher,genres,app_name,title,url,release_date,tags,reviews_url,specs,price,early_access,id,developer
13893,Bethesda Softworks,[Action],Wolfenstein II: The New Colossus,Wolfenstein II: The New Colossus,http://store.steampowered.com/app/612880/,2017-10-26,"[Action, FPS, Gore, Violent, Alternate History...",http://steamcommunity.com/app/612880/reviews/?...,"[Single-player, Steam Achievements, Full contr...",59.99,False,612880,Machine Games
14572,Bethesda Softworks,[Action],Wolfenstein II: The New Colossus,Wolfenstein II: The New Colossus,http://store.steampowered.com/app/612880/Wolfe...,2017-10-26,"[Action, FPS, Gore, Violent, Alternate History...",http://steamcommunity.com/app/612880/reviews/?...,"[Single-player, Steam Achievements, Full contr...",59.99,False,612880,Machine Games


Comprobamos que el `id` esta duplicado y procedemos a eliminarlo.

In [7]:
df_games.drop_duplicates(subset='id', inplace=True)
df_games.shape

(32132, 13)

#### 2.1.3 Renombramiento de la columna

Cambiamos el nombre de la columna `id` por `item_id` que es mas representativo.

In [8]:
df_games.rename(columns={'id': 'item_id'}, inplace=True)
df_games.columns

Index(['publisher', 'genres', 'app_name', 'title', 'url', 'release_date',
       'tags', 'reviews_url', 'specs', 'price', 'early_access', 'item_id',
       'developer'],
      dtype='object')

### 2.2 Columnas `title` y `app_name`

#### 2.2.1 Gestión de nulos

Notamos que ambas columnas contienen valores nulos que procedemos a reemplazar con 'unknown'.

In [13]:
columns = ['title', 'app_name']
df_games[columns] = df_games[columns].fillna('unknown')
df_games.isnull().sum()

publisher       8051
genres          3282
app_name           0
title              0
url                0
release_date    2066
tags             162
reviews_url        0
specs            669
price           1377
early_access       0
item_id            0
developer       3298
dtype: int64

#### 2.2.1 Verificación de duplicados

Exploramos el contenido de `title` y `app_name`.

In [14]:
df_games[columns]

,title,app_name
0,Lost Summoner Kitty,Lost Summoner Kitty
1,Ironbound,Ironbound
2,Real Pool 3D - Poolians,Real Pool 3D - Poolians
3,弹炸人2222,弹炸人2222
4,unknown,Log Challenge
...,...,...
32128,Colony On Mars,Colony On Mars
32129,LOGistICAL: South Africa,LOGistICAL: South Africa
32130,Russian Roads,Russian Roads
32131,EXIT 2 - Directions,EXIT 2 - Directions


A simple vista parece que `app_name` y `title` contienen los mismos datos, vamos a comprobarlo:

In [15]:
# Realizamos la comparación y creamos una columna con el resultado.
df_games['is_equal'] = df_games['app_name'] == df_games['title']

# Calculamos el porcentaje de filas donde 'app_name' y 'title' son iguales
equal_percentage = round((df_games['is_equal'].sum() / len(df_games)) * 100, 2)

print(f"El porcentaje de filas donde 'app_name' y 'title' son iguales es: {equal_percentage}%")

El porcentaje de filas donde 'app_name' y 'title' son iguales es: 91.9%


In [16]:
# Filtramos las filas donde 'app_name' y 'title' no son iguales.
not_equal_df = df_games.loc[df_games['is_equal'] == False]
not_equal_df[['app_name', 'title']]

,app_name,title
4,Log Challenge,unknown
11,Icarus Six Sixty Six,unknown
19,After Life VR,unknown
20,Kitty Hawk,unknown
22,Mortars VR,unknown
...,...,...
32071,Tank of War-VR,unknown
32074,Flappy Arms,unknown
32075,SpaceWalker,unknown
32083,LIV Client,unknown


Al comprobar que `app_name` y `title` son iguales en más del 91% y que el resto son datos faltantes, decidimos eliminar `title`, que tiene una mayor cantidad de datos faltantes.

In [17]:
df_games.drop(['title', 'is_equal'], axis=1, inplace=True)
df_games.columns

Index(['publisher', 'genres', 'app_name', 'url', 'release_date', 'tags',
       'reviews_url', 'specs', 'price', 'early_access', 'item_id',
       'developer'],
      dtype='object')

#### 2.2.3 Renombramiento de las columnas

Cambiamos el nombre de la columna `app_name` por uno mas representativo como `item_name`.

In [18]:
df_games.rename(columns={'app_name':'item_name'}, inplace=True)
df_games.columns

Index(['publisher', 'genres', 'item_name', 'url', 'release_date', 'tags',
       'reviews_url', 'specs', 'price', 'early_access', 'item_id',
       'developer'],
      dtype='object')

### 2.3 Columnas `publisher` y `developer`

#### 2.3.1 Gestión de nulos

Podemos eliminar la columna `publisher` que no es necesaria para los endpoints o preparar el modelo para el sistema de recomendación, y de esta manera optimizar el rendimiento de la API y el entrenamiento del modelo.

In [19]:
df_games.drop('publisher', axis=1, inplace=True)
df_games.columns

Index(['genres', 'item_name', 'url', 'release_date', 'tags', 'reviews_url',
       'specs', 'price', 'early_access', 'item_id', 'developer'],
      dtype='object')

Notamos que también tenemos valores nulos para la columna `developer`
que reemplazamos con 'unknown'.

In [20]:
df_games['developer'] = df_games['developer'].fillna('unknown')
df_games.isnull().sum()

genres          3282
item_name          0
url                0
release_date    2066
tags             162
reviews_url        0
specs            669
price           1377
early_access       0
item_id            0
developer          0
dtype: int64

### 2.4 Columna `release_date`





#### 2.4.1 Gestión de nulos

La variable `release_date` tiene **2066** datos faltantes. Reemplazamos por 'unknown'.

In [21]:
df_games['release_date'] = df_games['release_date'].fillna('unknown')
df_games.isnull().sum()

genres          3282
item_name          0
url                0
release_date       0
tags             162
reviews_url        0
specs            669
price           1377
early_access       0
item_id            0
developer          0
dtype: int64

#### 2.4.2 Extracción del año.

Para los endpoints que se consumirán en la API solo necesitamos el año de lanzamiento del item, para ello lo extraemos de `release_date` de la siguiente manera:

- Creamos una máscara booleana donde `release_date` no coincide con el formato `YYYY-MM-DD`

In [22]:
mask = df_games['release_date'].apply(lambda x: not re.match(r'\d{4}-\d{2}-\d{2}', str(x)))

 - Filtramos las filas donde la máscara es True

In [23]:
invalid_dates = df_games.loc[mask, 'release_date']
invalid_dates

4             unknown
10             Soon..
11            unknown
19            unknown
20            unknown
             ...     
32085     Coming Soon
32086         unknown
32095            2016
32121    January 2018
32132         unknown
Name: release_date, Length: 2351, dtype: object

- Extraemos el año en una nueva columna y si no existe reemplazamos por 'unknown'.

In [24]:
df_games['release_year'] = df_games['release_date'].str.extract(r'(\d{4})').fillna('unknown')

- Comprobamos que se haya extraido correctamente el año:

In [25]:
df_games[['release_date', 'release_year']].loc[[32121, 32095, 32085 ]]

,release_date,release_year
32121,January 2018,2018
32095,2016,2016
32085,Coming Soon,unknown


In [27]:
df_games['release_year'].unique()

array(['2018', '2017', 'unknown', '1997', '1998', '2016', '2006', '2005',
       '2003', '2007', '2002', '2000', '1995', '1996', '1994', '2001',
       '1993', '2004', '1999', '2008', '2009', '1992', '1989', '2010',
       '2011', '2013', '2012', '2014', '1983', '1984', '2015', '1990',
       '1988', '1991', '1985', '1982', '1987', '1981', '1986', '2021',
       '5275', '2019', '1975', '1970', '1980'], dtype=object)

Notamos que hay un año = '5275' lo cual es incongruente. Vamos a observarlo:

In [28]:
df_games[df_games['release_year'] == "5275"]

,genres,item_name,url,tags,reviews_url,specs,price,early_access,item_id,developer,release_year
13427,"[Casual, Indie, Early Access]",Puzzle Sisters Foer,http://store.steampowered.com/app/710190/Puzzl...,"[Early Access, Casual, Indie]",http://steamcommunity.com/app/710190/reviews/?...,"[Single-player, Steam Achievements, Steam Trad...",NaN,True,710190,一次元创作组,5275


Procedemos a reemplazarlo por 'unknown'.

In [29]:
df_games['release_year'] = df_games['release_year'].replace('5275', 'unknown')
df_games['release_year'].unique()

array(['2018', '2017', 'unknown', '1997', '1998', '2016', '2006', '2005',
       '2003', '2007', '2002', '2000', '1995', '1996', '1994', '2001',
       '1993', '2004', '1999', '2008', '2009', '1992', '1989', '2010',
       '2011', '2013', '2012', '2014', '1983', '1984', '2015', '1990',
       '1988', '1991', '1985', '1982', '1987', '1981', '1986', '2021',
       '2019', '1975', '1970', '1980'], dtype=object)

* Eliminamos `release_date` ya que no lo necesitamos.

In [26]:
df_games.drop('release_date', axis=1, inplace=True)
df_games.columns

Index(['genres', 'item_name', 'url', 'tags', 'reviews_url', 'specs', 'price',
       'early_access', 'item_id', 'developer', 'release_year'],
      dtype='object')

### 2.5 Columna `price`

#### 2.5.1 Gestión de nulos

Exploramos el contenido de la columna.

In [30]:
df_games['price'].head()

0            4.99
1    Free To Play
2    Free to Play
3            0.99
4            2.99
Name: price, dtype: object

Observamos que tiene cadenas de caracteres que hacen referencia a que el contenido es gratuito.

In [31]:
df_games[df_games['price'].isnull()][['price','genres', 'tags']].sample(10)

,price,genres,tags
499,NaN,[Action],"[Action, FPS]"
3840,NaN,"[Casual, Indie, Simulation, Strategy]","[Simulation, Strategy, Indie, Casual, Visual N..."
20356,NaN,"[Action, RPG]","[Action, RPG]"
27324,NaN,"[Adventure, Indie, RPG]","[Indie, Adventure, RPG]"
14700,NaN,"[Adventure, Casual, Indie, Simulation]","[Adventure, Simulation, Indie, Casual]"
2775,NaN,"[Adventure, Indie]","[Adventure, Indie, Multiplayer, Free to Play, ..."
24882,NaN,"[Indie, RPG]","[RPG, Indie]"
9969,NaN,"[Action, Indie, Massively Multiplayer, Simulat...","[Strategy, Action, Massively Multiplayer, Indi..."
18091,NaN,"[Action, Adventure, Indie]","[Adventure, Action, Indie, Parkour, 3D Platfor..."
13511,NaN,"[Action, Adventure, Indie]","[Action, Adventure, Indie]"


Observamos que tanto la columna `genres` como `tags` hacen referencia a que el contenido es gratuito. Por lo que podemos usarlas para imputar valores faltantes estableciendo `Price` en 0. Por lo tanto, creamos una función que busque la palabra **'free'** en las columnas `tags` y `genres`. Si encuentra que una cadena contiene la palabra **'free'** en alguna de estas columnas y el precio es **NaN**, debería establecer el precio en 0.

In [32]:
df_games.price.isnull().sum()

1377

In [33]:
def check_free(val):
    if isinstance(val, list):
        for item in val:
            if 'free' in item.lower():
                return True
    return False


# Aplicamos la función a cada fila del DataFrame
df_games['is_free'] = df_games['genres'].apply(check_free) | df_games['tags'].apply(check_free) | df_games['specs'].apply(check_free)

# Si un juego es gratuito y su precio es 'NaN', establece su precio en 0
df_games.loc[(df_games['is_free'] == True) & (df_games['price'].isnull()), 'price'] = 0

# Eliminamos la columna temporal 'is_free'
df_games.drop('is_free', axis=1, inplace=True)

Observamos que la cantidad de valores nulos ha disminuido, por lo que podemos concluir que teníamos varios juegos gratuitos con precios nulos, pero aún nos falta asignarle precio **0** a los juegos cuyo precio indica que es **'free'** en la columna `price`.

In [34]:
df_games.price.isnull().sum()

1172

Creamos otra función que busque la palabra **'free'** pero solamente en la columna `price`. Si alguna cadena contiene la palabra **'free'**, debería establecer el precio en **0**.

In [35]:
def replace_free_with_zero(val):
    if isinstance(val, str) and 'free' in val.lower():
        return 0
    return val

# Solo aplica la función a la columna 'price'
df_games['price'] = df_games['price'].apply(replace_free_with_zero)

In [36]:
df_games['price'].unique()

array([4.99, 0, 0.99, 2.99, 3.99, 9.99, 18.99, 29.99, nan, 10.99, 1.59,
       14.99, 1.99, 59.99, 8.99, 6.99, 7.99, 39.99, 19.99, 7.49, 12.99,
       5.99, 2.49, 15.99, 1.25, 24.99, 17.99, 61.99, 3.49, 11.99, 13.99,
       34.99, 74.76, 1.49, 32.99, 99.99, 14.95, 69.99, 16.99, 79.99,
       49.99, 5.0, 44.99, 13.98, 29.96, 119.99, 109.99, 149.99, 771.71,
       'Install Now', 21.99, 89.99, 'Play WARMACHINE: Tactics Demo', 0.98,
       139.92, 4.29, 64.99, 54.99, 74.99, 'Install Theme', 0.89,
       'Third-party', 0.5, 'Play Now', 299.99, 1.29, 3.0, 15.0, 5.49,
       23.99, 49.0, 20.99, 10.93, 1.39, 36.99, 4.49, 2.0, 4.0, 9.0,
       234.99, 1.95, 1.5, 199.0, 189.0, 6.66, 27.99, 10.49, 129.99, 179.0,
       26.99, 399.99, 31.99, 399.0, 20.0, 40.0, 3.33, 199.99, 22.99,
       320.0, 38.85, 71.7, 59.95, 995.0, 27.49, 3.39, 6.0, 19.95, 499.99,
       16.06, 4.68, 131.4, 44.98, 202.76, 1.0, 2.3, 0.95, 172.24, 249.99,
       2.97, 10.96, 10.0, 30.0, 2.66, 6.48, 19.29, 11.15, 18.9, 2.89,
  

Aún nos quedan cadenas que no contenían la palabra **'free'**, vamos a examinarlas para determinar si es contenido gratuito.

In [37]:
non_price_values = ['Install Now', 'Play WARMACHINE: Tactics Demo', 'Install Theme', 'Third-party', 'Play Now', 'Play the Demo', 'Starting at $499.00', 'Starting at $449.00']
df_games[df_games['price'].isin(non_price_values)]


,genres,item_name,url,tags,reviews_url,specs,price,early_access,item_id,developer,release_year
2404,[Utilities],EVGA Precision XOC,http://store.steampowered.com/app/268850/EVGA_...,"[Utilities, Software, Free to Play]",http://steamcommunity.com/app/268850/reviews/?...,"[Single-player, Steam Achievements]",Install Now,False,268850,EVGA,2014
2870,"[Indie, Strategy]",WARMACHINE: Tactics,http://store.steampowered.com/app/253510/WARMA...,"[Strategy, Turn-Based, Turn-Based Strategy, St...",http://steamcommunity.com/app/253510/reviews/?...,"[Single-player, Multi-player, Cross-Platform M...",Play WARMACHINE: Tactics Demo,False,253510,WhiteMoon Dreams,2014
3831,"[Adventure, Casual, Indie, RPG, Simulation]",FREE China Theme Pack,http://store.steampowered.com/app/370880/FREE_...,"[Adventure, RPG, Indie, Casual, Simulation]",http://steamcommunity.com/app/370880/reviews/?...,"[Single-player, Downloadable Content, Steam Ac...",Install Theme,False,370880,Stolen Couch Games,2015
3917,[Indie],Parcel - Soundtrack,http://store.steampowered.com/app/362970/Parce...,[Indie],http://steamcommunity.com/app/362970/reviews/?...,"[Single-player, Shared/Split Screen, Downloada...",Third-party,False,362970,Polar Bunny Ltd,2015
4025,"[Casual, Indie]",Oblivious Garden ~White Day,http://store.steampowered.com/app/345040/Obliv...,"[Casual, Indie]",http://steamcommunity.com/app/345040/reviews/?...,"[Single-player, Downloadable Content, Steam Ac...",Play Now,False,345040,"CorypheeSoft,DigitalEZ",2015
22733,[Strategy],Legends of Callasia,http://store.steampowered.com/app/438920/Legen...,"[Strategy, Wargame, Fantasy, Multiplayer, Turn...",http://steamcommunity.com/app/438920/reviews/?...,"[Single-player, Multi-player, Online Multi-Pla...",Play the Demo,False,438920,Boomzap Entertainment,2016
24999,NaN,Syber Steam Machine,http://store.steampowered.com/app/353420/Syber...,"[Steam Machine, Hardware]",http://steamcommunity.com/app/353420/reviews/?...,NaN,Starting at $499.00,False,353420,unknown,2015
25000,NaN,Alienware Steam Machine,http://store.steampowered.com/app/353390/Alien...,"[Steam Machine, Hardware, Gaming, Futuristic, ...",http://steamcommunity.com/app/353390/reviews/?...,NaN,Starting at $449.00,False,353390,unknown,2015
26216,"[Adventure, Casual, Indie, Simulation]",Area-X - Extra Gallery,http://store.steampowered.com/app/383860/AreaX...,"[Adventure, Indie, Casual, Simulation]",http://steamcommunity.com/app/383860/reviews/?...,"[Single-player, Downloadable Content]",Play Now,False,383860,Zeiva Inc,2015
31836,[Casual],Peggle Extreme,http://store.steampowered.com/app/3483/Peggle_...,"[Casual, Puzzle, Free to Play, Action]",http://steamcommunity.com/app/3483/reviews/?br...,[Single-player],Third-party,False,3483,"PopCap Games, Inc.",2007


Observamos dos registros con un precio inicial estipulado, el cual procedemos a establecer como su precio.

In [38]:
df_games.loc[df_games['price'] == 'Starting at $499.00', 'price'] = 499
df_games.loc[df_games['price'] == 'Starting at $449.00', 'price'] = 449

El resto de los registros parecen ser contenido extra como utilidades, temas, peliculas, entre otros. Los consideraremos como gratuitos.

In [39]:
non_price_values = ['Install Now', 'Play WARMACHINE: Tactics Demo', 'Install Theme', 'Third-party', 'Play Now', 'Play the Demo']
df_games.loc[df_games['price'].isin(non_price_values), 'price'] = 0

Calculamos la media, la moda y la mediana de la columna 'price' para imputar nulos.

In [40]:
media = df_games['price'].mean()
moda = df_games['price'].mode()[0]
mediana = df_games['price'].median()

print(f"La media de los precios es {media:.2f}")
print(f"La moda de los precios es {moda:.2f}")
print(f"La mediana de los precios es {mediana:.2f}")

La media de los precios es 8.84
La moda de los precios es 4.99
La mediana de los precios es 4.99


Los resultados obtenidos nos indican que los precios de los juegos tienen una distribución asimétrica a la derecha, es decir, que hay más valores bajos que altos. Esto se puede ver por el hecho de que la media (8.84) es mayor que la mediana (4.99) y la moda (4.99).

Para imputar los valores nulos, se podría considerar usar la mediana o la moda, ya que son más robustas a los valores extremos que la media.

Imputamos los valores faltantes con la mediana.

In [41]:
df_games['price'].fillna(df_games['price'].median(), inplace=True)

### 2.6 Columnas `genres`, `tags` y `specs`

#### 2.6.1 Gestión de nulos

Exploramos el contenido de 'genres', 'tags' y 'specs'.

In [42]:
columns = ['genres', 'tags','specs']
df_games[columns].sample(10)

,genres,tags,specs
18244,"[Indie, RPG, Strategy]","[Strategy, RPG, Indie]","[Multi-player, Co-op, Cross-Platform Multiplay..."
22069,NaN,"[Comedy, Sci-fi, Fantasy, Movie, Action-Advent...",[Captions available]
25834,"[Action, Adventure, Casual, Indie, RPG, Simula...","[RPG, Indie, Adventure, Simulation, RPGMaker, ...","[Single-player, Steam Achievements, Steam Trad..."
17590,[Action],"[Action, Hack and Slash, Historical, Anime]","[Single-player, Steam Achievements, Partial Co..."
24177,"[Action, Adventure, Indie]","[Indie, Adventure, Action, Puzzle, First-Perso...","[Single-player, Steam Achievements, Steam Trad..."
20692,NaN,"[Strategy, Casual, Simulation, VR]","[Single-player, HTC Vive, Tracked Motion Contr..."
22173,NaN,"[Casual, Simulation, VR]","[Single-player, HTC Vive, Tracked Motion Contr..."
22140,NaN,"[Early Access, Indie, Simulation, VR]","[Single-player, Steam Achievements, HTC Vive, ..."
23867,"[Adventure, Casual]","[Adventure, Casual, Hidden Object]","[Single-player, Steam Trading Cards]"
26697,[Adventure],"[Visual Novel, Mystery, Anime, Story Rich, Psy...","[Single-player, Steam Achievements, Steam Trad..."


Podemos observar que `tags` contiene tambien a `genres` por lo que podemos usarla para imputar los nulos en `genres`.

In [43]:
# Creamos una lista de géneros únicos
unique_genres = df_games['genres'].explode().unique()

# Definimos una función para buscar géneros en tags
def find_genres(row):
    if isinstance(row['tags'], list):
        genres_in_tags = [tag for tag in row['tags'] if tag in unique_genres]
        if genres_in_tags:
            return genres_in_tags
    return row['genres']

# Aplicamos la función a cada fila y almacenamos en la columna temporal 'genres_2'
df_games['genres_2'] = df_games.apply(find_genres, axis=1)
df_games[df_games['genres'].isnull()][['genres', 'genres_2', 'tags']].head()

,genres,genres_2,tags
4,NaN,"[Action, Indie, Casual, Sports]","[Action, Indie, Casual, Sports]"
11,NaN,[Casual],[Casual]
19,NaN,"[Early Access, Indie]","[Early Access, Indie, VR]"
20,NaN,"[Early Access, Action, Adventure, Indie, Casual]","[Early Access, Action, Adventure, Indie, Casual]"
22,NaN,"[Early Access, Strategy, Action, Indie, Casual]","[Early Access, Strategy, Action, Indie, Casual..."


Eliminamos la columnas `genres`, `tags` y `specs` que ya no necesitamos.

In [44]:
columns = ['genres', 'tags', 'specs']
df_games.drop(columns, axis=1, inplace=True)
# Renombramos la columna genres_2 como genres
df_games = df_games.rename(columns={'genres_2': 'genres'})
df_games.columns

Index(['item_name', 'url', 'reviews_url', 'price', 'early_access', 'item_id',
       'developer', 'release_year', 'genres'],
      dtype='object')

Por último, nos falta reemplazar los nulos restantes en nuestra nueva columnna `genres`.

In [45]:
df_games['genres'] = df_games['genres'].apply(lambda x: ['unknown'] if isinstance(x, float) else x)
df_games.isnull().sum()

item_name       0
url             0
reviews_url     0
price           0
early_access    0
item_id         0
developer       0
release_year    0
genres          0
dtype: int64

### 2.7 Columnas `url`, `reviews_url` y `early_access`.

Estas columnas también podemos eliminarlas ya que no se necesitan para responder las consultas de la API o preparar el modelo de aprendizaje automático, y de esta manera disminuir el almacenamiento utilizado en nuestra DB y optimizar el entrenamiento del modelo.

In [46]:
columns = ['url', 'reviews_url', 'early_access']
df_games.drop(columns, axis=1, inplace=True)
df_games.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 32132 entries, 0 to 32132
Data columns (total 6 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   item_name     32132 non-null  object 
 1   price         32132 non-null  float64
 2   item_id       32132 non-null  object 
 3   developer     32132 non-null  object 
 4   release_year  32132 non-null  object 
 5   genres        32132 non-null  object 
dtypes: float64(1), object(5)
memory usage: 2.7+ MB


## 3 Carga

Finalmente, en esta sección cargamos nuestros datos transformados a un destino interino para su posterior análisis y tratamiento mediante feature engineering. Optamos por almacenarlos en formato parquet con compresion snappy para reducir su tamaño de almacenamiento.

In [48]:
# Definimos la ruta de destino y el nombre del archivo.
folder_path = 'data/interim/'
file_name = 'steam_games.parquet'

# Verificamos si el folder_path existe
if not os.path.exists(folder_path):
    # Si no existe, lo creamos
    os.makedirs(folder_path)

# Exportamos el DataFrame a un archivo Parquet
df_games.to_parquet(os.path.join(folder_path, file_name), engine='pyarrow', compression='snappy')

print(f'El archivo {file_name} se guardó correctamente en {folder_path}')

El archivo steam_games.parquet se guardó correctamente en data/interim/


## 4 Referencias

* Steam store. (s/f). Steampowered.com. Recuperado el 25 de octubre de 2023, de https://store.steampowered.com/

* Steam web API. (s/f). Valvesoftware.com. Recuperado el 25 de octubre de 2023, de https://developer.valvesoftware.com/wiki/Steam_Web_API

* Steam community. (s/f). Steamcommunity.com. Recuperado el 25 de octubre de 2023, de https://steamcommunity.com/

